# Выбор подходящего тарифа для существующих клиентов

Мобильный оператор «Y-mobile» выяснил: значительная часть клиентов продолжает пользоваться архивными тарифами. Руководство поставило хадачу построения модели, которая будет способна сделать анализ поведения клиентов и предложить существующим клиентам с архивными тарифами новый тариф: «Смарт» или «Ультра».

## Рекомендация тарифов

Мы располагаем данными о поведении клиентов, которые уже перешли на эти тарифы. Небходимо создать модель для задачи классификации, которая выберет подходящий тариф.

Нужно построить модель с максимально большим значением *accuracy*. Целевое значение - 0.75.

## Описание данных 

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
 - сalls — количество звонков,
 - minutes — суммарная длительность звонков в минутах,
 - messages — количество sms-сообщений,
 - mb_used — израсходованный интернет-трафик в Мб,
 - is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

**Содержание проекта**
- [1. Изучение данных](#1.-Изучение-данных)
- [2. Разбивка данных на выборки](#2.-Разбивка-данных-на-выборки)
- [3. Исследование моделей](#3.-Исследование-моделей)
 - [3.1 Дерево решений](#3.1-Дерево-решений)
 - [3.2 Случайный лес](#3.2-Случайный-лес)
 - [3.3 Логистическая регрессия](#3.3-Логистическая-регрессия)
- [4. Проверка модели на тестовой выборке](#4.-Проверка-модели-на-тестовой-выборке)
- [5. (бонус) Проверка модели на адекватность](#5.-(бонус)-Проверка-модели-на-адекватность)

## 1. Изучение данных

Сначала импортируем необходимые нам для работы библиотеки и изученные в теоретическом блоке алгоритмы обучения.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.model_selection import train_test_split

Прочитаем файл и сохраним данные в датасет users.

In [2]:
users = pd.read_csv('/datasets/users_behavior.csv')

Посмотрим на первые пять строк.

In [3]:
users.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Пропусков в данных нет.

In [5]:
users.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


В описательной статистике таке все выглядит нормально.

Далее проверим данные на дубликаты.

In [6]:
duplicates = users[users.duplicated()]
duplicates

,calls,minutes,messages,mb_used,is_ultra


Дубликатов нет.

Наконец посмотрим на соотношение значений целевого признака.

In [7]:
users['is_ultra'].value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

Получается, что у нас только около 30.5% абоннетов пользуются тарифом "Ультра". То есть, мы должны обратить внимание на тот факт, что если в будущем точность нашей модели будет около 70% - это может означать, что она не способна предсказывать тариф "Ультра".

Перейдем к разделению данных на выборки.

## 2. Разбивка данных на выборки

Нам необходимо выделить среди наших данных три выборки - обучающую, валидационную и тестовую.

Мы возьмем их в соотношениях 3 : 1 : 1.

Для этого воспользуемся функцией train_test_split 2 раза, кроме этого, сделаем стратификацию, чтобы доли обоих значений целевого признака во всех выборках были равны.

In [8]:
target = users['is_ultra']
features = users.drop('is_ultra', axis=1)

features_train, features_valid_test, target_train, target_valid_test = train_test_split(features, 
                                        target, test_size=0.4, random_state=1, stratify = target)

features_valid, features_test, target_valid, target_test = train_test_split(features_valid_test, 
                        target_valid_test, test_size=0.5, random_state=1, stratify=target_valid_test)

Итак, данные разделены на выборки; признаки и целевые признаки выделены.

Проверим, представительство классов в выборках.

In [9]:
for i in [target_train, target_valid, target_test]:
    display(i.value_counts(), i.value_counts(normalize=True))     

0    1337
1     591
Name: is_ultra, dtype: int64

0    0.693465
1    0.306535
Name: is_ultra, dtype: float64

0    446
1    197
Name: is_ultra, dtype: int64

0    0.693624
1    0.306376
Name: is_ultra, dtype: float64

0    446
1    197
Name: is_ultra, dtype: int64

0    0.693624
1    0.306376
Name: is_ultra, dtype: float64

Заметно, что стратификация прошла успешно и доли классов во всех трех выборках одинаковы.

Итак, выборки сформированы в соотношении 3 : 1 : 1, признакам присвоены имена. Можно приступать к исследованию моделей.

## 3. Исследование моделей

Прежде чем переходить к изучению моделей, создадим функцию для проверки точности модели на гиперпараметрах по умолчанию.

In [10]:
# в качестве аргументов передается алгоритм обучения, 
# признаки и целевые признаки обучающей и валидационной выборок
# модель обучается, делает прогноз, и затем вызывается функция проверки точности

def default_check(model_type, f_train, t_train, f_valid, t_valid):
    model = model_type(random_state=1)
    model.fit(f_train, t_train)
    model_predictions = model.predict(f_valid)
    return accuracy_score(t_valid, model_predictions)

### 3.1 Дерево решений

Первой рассмотренной моделью будет алгоритм дерева решений.

Вызовем для него проверку по умолчанию.

In [11]:
default_check(DecisionTreeClassifier, features_train, target_train, features_valid, target_valid)

0.7013996889580093

Получается, что на валидационной выборке наша модель показала точность чуть выше 70%, что ниже требуемого заданием порога в 75%.

Посмотрим, приведет ли изменение гиперпараметров к повышению точности.

Перебирать различные значения гиперпараметров будем во вложенных циклах.

Для модели дерева решения попробуем менять следующие параметры (здесь и далее описание гиперпараметров приведено как попытка перевода с английского из документаций соответствующих алгоритмов):

 - максимальная глубина дерева;
 
 
 - минимальное количество объектов в листе - он будет запрещать создавать лист, в котором слишком мало объектов обучающей выборки, по умолчанию = 1;
 
 
 - минимальное количество примеров для разделения - будет запрещать создавать узлы, в которые попадает слишком мало объектов обучающей выборки, по умолчанию = 2.

Перед выполнением цикла создадим новые переменные, которые будет сохранять информацию о гиперпараметрах лучшей по точности модели.

После обучения модели будем проверять ее на валидационной выборке.

In [12]:
best_tree_model = None
best_tree_result = 0
best_depth = 0
best_samples_leaf = 0
best_samples_split = 0

for depth in range(1,10):
    for leaf in range(1,20):
        for split in range(2,10):
            tree_model = DecisionTreeClassifier(random_state=1, max_depth=depth, 
                                min_samples_leaf=leaf, min_samples_split=split)
            
            tree_model.fit(features_train, target_train)
            tree_predictions = tree_model.predict(features_valid)
            result = accuracy_score(target_valid, tree_predictions)
            
            if result > best_tree_result:
                
                best_tree_model = tree_model
                best_tree_result = result
                best_depth = depth
                best_samples_leaf = leaf
                best_samples_split = split


Итак, выведем точность и значения гиперпараметров алгоритма лучшей модели.

In [13]:
print(f'Точность наилучшей модели дерева решений: {best_tree_result:.2%}')
print(f'Максимальная глубина: {best_depth}')
print(f'Минимальное количество объектов в листе: {best_samples_leaf}')
print(f'Минимальное количество примеров для разделения: {best_samples_split}')

Точность наилучшей модели дерева решений: 80.40%
Максимальная глубина: 7
Минимальное количество объектов в листе: 6
Минимальное количество примеров для разделения: 2


Получается, что точность модели выросла более чем на 10%, по сравнению с настройками по умолчанию.

Мы видим, что наше лучшее дерево достигло глубины в 7 уровней. Значение по умолчанию для этого гиперпараметра отсутствует - узлы расширяются до тех пор, пока все листья - чистые или содержат меньше, чем минимальное количество примеров для разделения.

Кроме того, в лучшей модели минимальное количество объектов выборки, попадающих в лист - 6.

Минимальное количество примеров для разделения осталось значением "по умолчанию".

### 3.2 Случайный лес

Далее рассмотрим модель случайного леса.

Проверим ее на гиперпараметрах по умолчанию.

In [14]:
default_check(RandomForestClassifier, features_train, target_train, features_valid, target_valid)

/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.7838258164852255

Модель сразу же выдала точность выше 78%.

Посомтрим, можно ли ее повысить, изменяя гиперпараметры.

В качестве изменяемых гиперпараметров возьмем следующие:
 - n_estimators - количество деревьев в лесу;
 - максимальная глубина дерева;
 - максимальное количество листьев - то есть конечных ячеек.

Выполним цикл для случайного леса.

In [15]:
best_forest_model = None
best_forest_result = 0
best_depth_for_forest = 0
best_estimators = 0
best_max_leaf_nodes = 0

for est in range(1,10):
    for depth in range(1,10):
        for node in range(2,10):
            forest_model = RandomForestClassifier(random_state=1, max_depth=depth, 
                                n_estimators=est,max_leaf_nodes=node)
            
            forest_model.fit(features_train, target_train)
            forest_predictions = forest_model.predict(features_valid)
            result = accuracy_score(target_valid, forest_predictions)
            
            if result > best_forest_result:
                
                best_forest_model = forest_model
                best_forest_result = result
                best_depth_for_forest = depth
                best_estimators = est
                best_max_leaf_nodes = node

Выведем значения точности и гиперпараметров.

In [16]:
print(f'Точность наилучшей модели случайного леса: {best_forest_result:.2%}')
print(f'Максимальная глубина: {best_depth_for_forest}')
print(f'Количество деревьев: {best_estimators}')
print(f'Максимальное количество листьев: {best_max_leaf_nodes}')

Точность наилучшей модели случайного леса: 80.09%
Максимальная глубина: 3
Количество деревьев: 4
Максимальное количество листьев: 7


Итак, лучшая модель случайного леса насчитывает в себе 4 дерева.

Нам незначительно удалось повысить показатель точности, изменняя гиперпараметры.

То есть, алгоритм обучил 4 независимых друг от друга случайных деревьев, а потом принял решение на основе голосования.

Максимальная глубина - 3, у этой модели уровней у дерева меньше, чем у предыдущей.

Кроме того, получилось, что у лучшей модели число конечных ячеек (листьев) ограничено семью. (В этих ячейках назначаются классы).

### 3.3 Логистическая регрессия

Далее рассмотрим алгоритм логистической регрессии.

Выполним для него проверку по умолчанию.

In [17]:
default_check(LogisticRegression, features_train, target_train, features_valid, target_valid)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7511664074650077

Точность модели на валидационной выборке - около 75%.

Посмотрим, удастся ли его повысить меняя гиперпараметры.

Попробуем менять следующие:
 - включение константы (пересечения с осью y) в функцию (по умолчанию=True);
 - выбор алгоритма для решения задачи оптимизации (по умолчанию lbfgs - метод с ограничением использования памяти алгоритма Бройдена — Флетчера — Гольдфарба — Шанно )
 - попробуем разное количество итераций для алгоритма оптимизации - при малых значений модель предупреждала об ошибке.

In [18]:
best_LR_model = None
best_LR_result = 0
best_intercept = 0
best_solver = None
best_iter = 0
best_class = None

for intercept in [True, False]:
    for solver in ['lbfgs', 'liblinear', 'sag', 'saga']:
        for iterations in range(4000, 7000, 1500):
             
                    LR_model = LogisticRegression(random_state=1, fit_intercept=intercept,
                                             solver=solver, max_iter=iterations)
            
                    LR_model.fit(features_train, target_train)
                    LR_predictions = LR_model.predict(features_valid)
                    result = accuracy_score(target_valid, LR_predictions)
            
                    if result > best_LR_result:
                        best_LR_model = LR_model
                        best_LR_result = result
                        best_intercept = intercept
                        best_solver = solver
                        best_iter = iterations

In [19]:
print(f'Точность наилучшей логистической регрессии: {best_LR_result:.2%}')
print(f'Включаем ли константу в функцию: {best_intercept}')
print(f'Алгоритм для решения задачи оптимизации: {best_solver}')
print(f'Максимальное количество итераций для алгоритма оптимизации: {best_iter}')

Точность наилучшей логистической регрессии: 75.27%
Включаем ли константу в функцию: True
Алгоритм для решения задачи оптимизации: lbfgs
Максимальное количество итераций для алгоритма оптимизации: 4000


Итак, получается, что точность модели после манипуляций на валидационной выборке незначительно выросла.

Включение константы в функцию в лучшей модели осталось значением по умолчанию (включаем).

Алгоритм оптимизации остался также по умолчанию.

А количество итераций, потребовавшееся, чтобы модель работала без предупреждений - 4000.

### Вывод 

Итак, мы рассмотрели три алгоритма обучения, для каждого из которых меняли гиперпараметры (в количестве трех штук).

В целом, можно сказать, что изменение гиперпараметров положительно сказалось на повышении точности моделей на валидационной выборке, по сравнению с точностью при использовании алгоритмами гиперпараметров по умолчанию.

Для дерева решений рост точности был заметным. Для случайного леса  - более слабым. Для логистической регрессии - почти незаметным.

В целом можно сказать, что затратив время на подбор лучших гиперпараметров, нам удалось добиться роста точности моделей на валидационной выборке.

## 4. Проверка модели на тестовой выборке

Попробуем проверить точность моделей на тестовой выборке.

Для этого напишем функцию проверки точности, которая принимает модели и тестовую выборку.

In [20]:
def test_accuracy(model, features, target):
    predictions = model.predict(features)
    return accuracy_score(target, predictions)

In [21]:
models = {'Дерево решений': best_tree_model, 'Случайный лес' : best_forest_model, 
          'Логистическая регрессия': best_LR_model}
for name, model in models.items():
    print(f'Точность модели {name} на тестовой выборке: {test_accuracy(model, features_test, target_test):.2%}')

Точность модели Дерево решений на тестовой выборке: 79.63%
Точность модели Случайный лес на тестовой выборке: 80.25%
Точность модели Логистическая регрессия на тестовой выборке: 75.74%


Далее воспользуемся метрикой recall_score, чтобы узнать, какую долю каждого класса наши модели предсказали правильно.

In [22]:
from sklearn.metrics import recall_score

In [23]:
for name, model in models.items():
    print(f'recall_score {name} на тестовой выборке: {recall_score(target_test, model.predict(features_test), average=None)}')

recall_score Дерево решений на тестовой выборке: [0.95515695 0.43654822]
recall_score Случайный лес на тестовой выборке: [0.93721973 0.49746193]
recall_score Логистическая регрессия на тестовой выборке: [0.98206278 0.24873096]


### Вывод

Итак, мы решали задачу классификации абонентов, исходя из того, сколько звонков делают, сколько минут они говорят, сколько сообщений отправляют и сколько мегабайтов интернета в месяц они тратят, зная пользуются ли они тарифом ultra, либо тарифом smart.

Все данные мы разбили на три выборки - обучающую, валидационную и тестовую в отношении 3 : 1 : 1. 

Модели мы обучали тремя алгоритмами обучения - деревом решений, случайным лесом и логистической регрессией.

Все модели после перебора индивидуальных гиперпараметров показали повышение точности на валидационной выборке по сравнению с гиперпараметрами по умолчанию.

На тестовой выборке Дерево решений и случайный лес показали точность около 80% соответственно. В то время как логистическая регрессия чуть перешагнула за границу в 75%.

Также интересны были результаты работы recall_score. Оказалось, что случайный лес хуже всех прогнозирует абонентов тарифа "Смарт" (93%), но лучше всех - абонентов 'ultra' (почти 50%). Дерево решений имеет похожие результаты, но немного лучше по "смарт" и существенно хуже по "ультра".

Логистическая регрессия отлично предсказывает абонентов "смарт" и только четверть абонентов "ультра".

## 5. (бонус) Проверка модели на адекватность

Итак, sanity-check или "проверка на вменяемость" подразумевает относительно простой тест для определения, может ли быть модель адекватно применяемой.

Для того, чтобы попытаться провести такую проверку для наших моделей обратимся к значениям целевого признака по датасету.

In [24]:
users['is_ultra'].mean()

0.30647168637212197

Значит, в нашем наборе данных около 69.5% абонентов пользуются тарифом "смарт".

Следовательно, если мы предположим, что наши модели будут всегда предсказывать тариф "смарт", то точность моделей была бы на уровне 69.5%. Но у каждой из трех моделей после подбора лучших гиперпараметров она превышает это значение.

Таким обазом, на мой взгляд, модели прошли проверку на вменяемость.

In [25]:
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(features_train, target_train)
dummy_clf.score(dummy_clf.predict(features_valid), target_valid)

0.6936236391912908